# Задание 3

- ### *Подключитесь к своей БД через Jupyter Notebook (используйте библиотеку pymysql)*

Импортируем все необходимые зависимости

In [1]:
import pymysql
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd

Ищем файл .env с переменными окружения и загружаем их

In [ ]:
load_dotenv()

Подключаемся к БД функцией connect, указав параметры подключения из переменных окружения

In [ ]:
connection = pymysql.connect(
    host=os.getenv("DB_HOST"),
    port=int(os.getenv("DB_PORT")),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME"),
    cursorclass=pymysql.cursors.DictCursor,
)

connection

- ### *Дальше используйте метод cursor для дальнейших манипуляций, создайте через jupyter новую БД*

Создадим методом cursor.execute с соответствующей SQL-командой новую базу данных

In [4]:
cursor = connection.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS new_database;")
connection.commit()

- ### *Используйте USE (опишите зачем нужен этот метод)*

Переключим активную БД командой USE, с которой будут выполняться все дальнейшие SQL-операции

In [ ]:
cursor.execute("USE new_database;")

- ### *Затем создайте таблицу с небольшим количеством полей и заполните ее данными*

Используем команду CREATE TABLE в методе execute для создания структуры таблицы и INSERT INTO в методе execute для добавления строк в таблицу

In [6]:
cursor.execute(
    """
    CREATE TABLE students (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(50),
        age INT
    );
"""
)

cursor.execute(
    """
    INSERT INTO students (name, age) VALUES
    ('Aslan', 23),
    ('Vyacheslav', 22),
    ('Maxim', 22);
"""
)

connection.commit()

Убедимся, что данные добавлены

In [ ]:
cursor.execute(
    """
    SELECT * FROM students;
"""
)

results = cursor.fetchall()

for row in results:
    print(row)

connection.commit()

- ### *Затем создайте новую таблицу через метод to.sql. В этой таблице должны быть перенесены данные с DataFrame из полученного DataFrame из 1го задания*

Создадим датафрейм из CSV-таблицы

In [ ]:
df_words = pd.read_csv("data/words.csv")

df_words

Создадим объект engine

In [9]:
engine = create_engine(
    f"mysql+pymysql://{os.getenv("DB_USER")}:{os.getenv("DB_PASSWORD")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/new_database"
)

Запишем данные из датафрейма в таблицу SQL-базы данных

In [ ]:
df_words.to_sql("words", engine, if_exists="replace", index=False)

- ### *В конечном итоге выведите 2 первые строки из таблицы созданной на основе DataFrame*

Получим данные из БД и выведем их в консоль с помощью метода read_sql 

In [ ]:
pd.read_sql("SELECT * FROM words LIMIT 2;", engine)

Закрытие соединения и очистка ресурсов

In [12]:
cursor.close()
connection.close()
engine.dispose()